# Анализ данных Всемирной организации здравохранения


## Часть 2. Получение данных по определенному индикатору

**Введение**<br>
В первой части мы загрузили Размерности и Индикаторы (Dimension и Indicator соответственно) в соответствующий файл Excel.<br>
Открыв этот файл, можно увидеть громадное количество Индикаторов.
В этом документe Notebook будет рассмотрен *"Life expectancy at birth"*. Ему соответствует код *"WHOSIS_000001"*, который и будет использован для запроса с сайта ВОЗ.

Загружаем необходимые модули.

In [1]:
import pandas as pd
import requests

В переменной *name* указываем интересующий нас индикатор (в нашем случае "Life expectancy at birth").<br>
В *file_name* определяется название Excel-файла, в который мы будем записывать полученные данные.<br>
Затем в переменную *indicator_code* записываем код нашего индикатора.<br>
(Полный список Индикатор и соответствующих им кодов представлен в файле "who_begin.xlsx".)

In [2]:
# Список перемееных
name = "Life expectancy at birth"
file_name = "{}.xlsx".format(name)
#sheet_name = "Life expectancy at birth"
indicator_code = "WHOSIS_000001"

url_indicator = "https://ghoapi.azureedge.net/api/{}".format(indicator_code)

**Функция для получения данных.<br>**
Как и в случае получения списков Размерностей и Индикатор, необходимые данные хранятся в ключе *value*. 
Далее делаем запрос. Сохраняем данные в dataframe, удаляя столбцы, в которых нет значений.

И, наконец, выводим первые пять строчек.<br>
(В столбце *Comments* есть не все значения, поэтому у первых 5 строк указано None).

In [3]:
# Функция для получения данных с сайта ВОЗ
def retrieving_data(url):
    request = requests.get(url)
    json_response = request.json()

    return json_response["value"]

In [4]:
# Совершаем запрос данных
indicator = retrieving_data(url_indicator)
df_indicator = pd.DataFrame(indicator).dropna(axis=1, how='all')
df_indicator.head()

,Comments,Date,Dim1,Dim1Type,Id,IndicatorCode,NumericValue,SpatialDim,SpatialDimType,TimeDim,TimeDimType,Value
0,None,2017-03-31T08:14:36.203+02:00,MLE,SEX,15578241,WHOSIS_000001,42.99474,RWA,COUNTRY,2000,YEAR,43.0
1,None,2017-03-31T08:14:36.36+02:00,FMLE,SEX,15578243,WHOSIS_000001,48.38779,RWA,COUNTRY,2000,YEAR,48.4
2,None,2017-03-31T08:14:36.453+02:00,BTSX,SEX,15578245,WHOSIS_000001,45.70058,RWA,COUNTRY,2000,YEAR,45.7
3,None,2017-03-31T08:14:36.58+02:00,MLE,SEX,15578247,WHOSIS_000001,43.48892,RWA,COUNTRY,2001,YEAR,43.5
4,None,2017-03-31T08:14:36.687+02:00,FMLE,SEX,15578249,WHOSIS_000001,49.49390,RWA,COUNTRY,2001,YEAR,49.5


**Запись в эксель.<br>**
Для записи в эксель используется *ExcelWriter* с движком *xlsxwriter*.

Сначала создаем экземпляр Pandas Excel writer и записываем данные по Индикатру в Excel.

Затем делаем небольшое форматирование, чтобы легче было просматривать получившийся файл.

И закрываем экземпляр Pandas Excel writer

In [6]:
# Создание экземпляра Pandas Excel writer
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

# Конвертирование датафрейма в XlsxWriter Excel объект
# Convert the dataframe to an XlsxWriter Excel object.
df_indicator.to_excel(writer)

# Получение xlsxwriter объекта writer.book
# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book

# Получение xlsxwriter объекта writer.sheets
# Get the xlsxwriter objects from the dataframe writer object.
worksheet = writer.sheets['Sheet1']
worksheet.set_column("A:A", 5)
worksheet.set_column("B:B", 40)
worksheet.set_column("C:C", 10)

# Закрытие Pandas Excel writer и запись в Эксель файл
# Close the Pandas Excel writer and output the Excel file.
writer.save()

**Окончание.**<br>
В получившимся Excel файле (название файла указана в переменной *file_name*) содержатся данные по всем странам по указанному Индикатору. Эти данные обрабатываются уже в последней 3-ей части.